<a href="https://colab.research.google.com/github/aplaza2/IIC3697-Proyecto/blob/main/ingles_kiche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traducción Inglés -> Kiche


## Configuración inicial

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# ===================================================================
# CELDA DE CONFIGURACIÓN CENTRAL DEL PROYECTO
# ===================================================================
# Para cambiar de experimento, solo necesitan modificar esta celda.

import os

# --- 1. Definición de la Tarea de Traducción ---
# Idioma de origen (el que no es español)
SOURCE_LANG_NAME = "Inglés"
SOURCE_LANG_CODE = "eng_Latn"

# Idioma de destino (la lengua maya que están estudiando)
TARGET_LANG_NAME = "K'iche'"
TARGET_LANG_CODE = "quc_Latn" # Código correcto para K'iche'
TARGET_LANG_SHORT_CODE = "quc" # Código del dataset MayanV

# Idioma pivote (el que se usa como puente)
PIVOT_LANG_NAME = "Español"
PIVOT_LANG_CODE = "spa_Latn"
PIVOT_LANG_SHORT_CODE = "es" # Código del dataset MayanV

# --- 2. Parámetros del Modelo y Entrenamiento ---
BASE_MODEL_ID = "facebook/nllb-200-distilled-600M"

# --- 3. Rutas para Guardar en Google Drive ---
# Asegúrate de que esta carpeta exista en tu Google Drive
DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion"
ES_QUC_DATASET_PATH = f"{DRIVE_PATH}/es_quc_dataset"
MAYANV_REPO_PATH = f"{DRIVE_PATH}/mayanv"
EN_QUC_DATASET_FILES_PATH = f"{DRIVE_PATH}/en_quc_dataset_files"
os.makedirs(DRIVE_PATH, exist_ok=True) # Crea la carpeta si no existe

# Nombres de los modelos que se guardarán
FINETUNED_PIVOT_MODEL_PATH = f"{DRIVE_PATH}/nllb_finetuned_{PIVOT_LANG_SHORT_CODE}_{TARGET_LANG_SHORT_CODE}"
FINETUNED_SYNTHETIC_MODEL_PATH = f"{DRIVE_PATH}/nllb_finetuned_{SOURCE_LANG_NAME.lower()}_{TARGET_LANG_SHORT_CODE}"

print(f"✅ Configuración cargada para la tarea: {SOURCE_LANG_NAME} -> {TARGET_LANG_NAME}")
print(f"Los modelos se guardarán en: {DRIVE_PATH}")

✅ Configuración cargada para la tarea: Inglés -> K'iche'
Los modelos se guardarán en: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion


### Funciones comunes

In [47]:
def translate_text(model, tokenizer, text, src_lang, tgt_lang):
    tokenizer.src_lang = src_lang
    forced_bos_id = tokenizer.convert_tokens_to_ids(tgt_lang)
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_id)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [48]:
def generate_dataset(language, train_folder="train", test_folder="test", base_path=MAYANV_REPO_PATH): # Use MAYANV_REPO_PATH from config
    """
    Generates a dataset from text files within the mayanv repository.

    Args:
        language (str): The language code (e.g., "quc").
        train_folder (str): The name of the training data folder within the language directory.
        test_folder (str): The name of the testing data folder within the language directory.
        base_path (str): The base path to the cloned mayanv repository in Google Drive.

    Returns:
        DatasetDict: A DatasetDict containing the train and test splits.
    """
    # Construct paths to the text files within the mayanv repository in Drive
    mayanv_language_path = os.path.join(base_path, "MayanV", language)
    train_lang_path = os.path.join(mayanv_language_path, train_folder, f"data.{language}")
    test_lang_path = os.path.join(mayanv_language_path, test_folder, f"data.{language}")

    train_es_path = os.path.join(mayanv_language_path, train_folder, "data.es")
    test_es_path = os.path.join(mayanv_language_path, test_folder, "data.es")

    print(f"Attempting to load train split from: {train_es_path} and {train_lang_path}")
    print(f"Attempting to load test split from: {test_es_path} and {test_lang_path}")


    # Carga manual de archivos
    def load_lines(path):
        if not os.path.exists(path):
            print(f"❌ Error: File not found at {path}")
            return None
        with open(path, encoding="utf-8") as f:
            return [line.strip() for line in f if line.strip()] # Filter out empty lines


    train_src = load_lines(train_lang_path)
    train_tgt = load_lines(train_es_path)

    test_src = load_lines(test_lang_path)
    test_tgt = load_lines(test_es_path)

    # Check if loading was successful
    if train_src is None or train_tgt is None or test_src is None or test_tgt is None:
        return None

    # Crea datasets
    train_dataset = Dataset.from_dict({"input": train_src, "target": train_tgt})
    test_dataset = Dataset.from_dict({"input": test_src, "target": test_tgt})

    return DatasetDict({
        "train": train_dataset,
        "test": test_dataset,
    })

## Librerías

In [49]:
from IPython.display import clear_output
!pip install -U bitsandbytes accelerate peft transformers sentencepiece datasets
!pip install evaluate
!pip install rouge_score
clear_output()

In [50]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset, DatasetDict, Dataset
import evaluate
from tqdm import tqdm
import csv
import torch

In [51]:
from peft import PeftModel

In [52]:
!git clone https://github.com/transducens/mayanv.git

fatal: destination path 'mayanv' already exists and is not an empty directory.


## Datasets

### Verificacion inicial de lenguaje en el modelo NLLB

In [117]:
from transformers import AutoTokenizer

# Cargamos el tokenizador oficial de NLLB
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# --- Verifiquemos los idiomas ---

kiche_code = "quc_Latn"
sipakapense_code = "qum_Latn"

print(f"Buscando el ID para K'iche' ({kiche_code})...")
try:
    kiche_id = tokenizer.convert_tokens_to_ids(kiche_code)
    print(f"✅ ÉXITO: El ID para '{kiche_code}' es: {kiche_id}. ¡SÍ EXISTE!")
except Exception as e:
    print(f"❌ FALLO: No se pudo encontrar un ID para '{kiche_code}'. ¡NO EXISTE!")

print("-" * 20)

print(f"Buscando el ID para Sipakapense ({sipakapense_code})...")
try:
    # El tokenizador arroja un error si no encuentra el token.
    # Usamos .get para manejarlo sin que el programa se caiga.
    sipakapense_id = tokenizer.vocab.get(sipakapense_code)
    if sipakapense_id is not None:
         print(f"✅ ÉXITO: El ID para '{sipakapense_code}' es: {sipakapense_id}. ¡SÍ EXISTE!")
    else:
         print(f"❌ FALLO: No se encontró un ID para '{sipakapense_code}'. ¡NO EXISTE!")
except Exception as e:
    print(f"❌ FALLO: Ocurrió un error buscando '{sipakapense_code}'.")

Buscando el ID para K'iche' (quc_Latn)...
✅ ÉXITO: El ID para 'quc_Latn' es: 3. ¡SÍ EXISTE!
--------------------
Buscando el ID para Sipakapense (qum_Latn)...
❌ FALLO: No se encontró un ID para 'qum_Latn'. ¡NO EXISTE!


### Siguientes pasos

Dataset se puede encontrar en el [siguiente enlace](https://github.com/transducens/mayanv).

In [ ]:
es_quc_dataset = generate_dataset("quc", train_folder="train", test_folder="test")
es_quc_dataset

Attempting to load train split from: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/mayanv/MayanV/quc/train/data.es and /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/mayanv/MayanV/quc/train/data.quc
Attempting to load test split from: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/mayanv/MayanV/quc/test/data.es and /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/mayanv/MayanV/quc/test/data.quc


DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 624
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 1000
    })
})

In [ ]:
# Save the dataset to the drive path
es_quc_dataset.save_to_disk(f"{DRIVE_PATH}/es_quc_dataset")

# Save the cloned git repository to the drive path
!cp -r mayanv/ "{DRIVE_PATH}/mayanv"


Saving the dataset (0/1 shards):   0%|          | 0/624 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

^C


## Preparación y Entrenamiento de Modelos


### 1. Modelo Zero-shot

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
zeroshot_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
zeroshot_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
zeroshot_tokenizer.src_lang = "eng_Latn"
forced_bos_id = zeroshot_tokenizer.convert_tokens_to_ids("quc_Latn")

### 2. Modelo con Finetuning Español-Kiche

Aquí entrenamos un modelo para que sea un experto en una tarea específica: Español -> K'iche'. Este modelo se usará después como una pieza en el experimento del pipeline.

#### 2.1 Modelo con Finetuning Español-Kiche

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
finetune_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


finetune_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Configurar LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Aplicar QLoRA
finetune_model = get_peft_model(finetune_model, lora_config)

In [ ]:
def preprocess_function(examples):
    max_length = 128
    # Tokeniza el input
    inputs = finetune_tokenizer(examples["input"], max_length=max_length, truncation=True, padding=True)
    # Tokeniza la salida (target)
    targets = finetune_tokenizer(examples["target"], max_length=max_length, truncation=True, padding=True)

    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_train = es_quc_dataset["train"].map(preprocess_function, batched=True)
tokenized_test = es_quc_dataset["test"].map(preprocess_function, batched=True)

Map:   0%|          | 0/624 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./qlora_finetuned_qum_nllb",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs_qlora",
    save_total_limit=2,
    bf16=False,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=finetune_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=finetune_tokenizer,
)

trainer.train()

# Define the save path
save_path = f"{DRIVE_PATH}/qlora_finetuned_es_quc_nllb"

# Save the model and tokenizer to the specified path
finetune_model.save_pretrained(save_path)
finetune_tokenizer.save_pretrained(save_path)

print(f"✅ Model and tokenizer saved to {save_path}")

/tmp/ipython-input-16-279613285.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,5.500633
2,No log,5.436388
3,No log,5.419795


✅ Model and tokenizer saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/qlora_finetuned_es_quc_nllb


### 3. Modelo con Finetuning Inglés-Kiche

#### 3.1 CREACIÓN DEL DATASET SINTÉTICO usando el modelo Zero-Shot

Tenemos que fábricar las palabras nuevas, convertir un par (Español, K'iche') en un par (Inglés, K'iche'). Para esto usamos el modelo Zero-Shot para obtener las traducciones pertinentes

In [ ]:
def build_en_quc_split(split, model, tokenizer, dataset):

    print(dataset["train"][8])
    new_inputs = []
    new_targets = []

    print(f"🔄 Traduciendo split '{split}' de español a inglés...\n")
    for example in tqdm(dataset[split]):
        espanol_text = example["target"]
        kiche_text = example["input"]
        english_text = translate_text(model, tokenizer, espanol_text, "spa_Latn", "eng_Latn")
        new_inputs.append(english_text)
        new_targets.append(kiche_text)

    return Dataset.from_dict({
        "input": new_inputs,
        "target": new_targets
    })

# Generar splits
dataset_variable_name = f"es_{TARGET_LANG_SHORT_CODE}_dataset"
es_target_dataset = globals().get(dataset_variable_name)

if es_target_dataset is None:
    raise NameError(f"Dataset variable '{dataset_variable_name}' not found. Please ensure the dataset for {PIVOT_LANG_NAME}-{TARGET_LANG_NAME} is generated and named correctly.")


en_quc_train = build_en_quc_split("train", zeroshot_model, zeroshot_tokenizer, es_target_dataset)
en_quc_test = build_en_quc_split("test", zeroshot_model, zeroshot_tokenizer, es_target_dataset)

# Combinar
en_quc_dataset = DatasetDict({
    "train": en_quc_train,
    "test": en_quc_test
})

{'input': 'Ri tat Xwan potzʼ', 'target': '#quc# Don Juan está ciego'}
🔄 Traduciendo split 'train' de español a inglés...



100%|██████████| 624/624 [25:55<00:00,  2.49s/it]


{'input': 'Ri tat Xwan potzʼ', 'target': '#quc# Don Juan está ciego'}
🔄 Traduciendo split 'test' de español a inglés...



100%|██████████| 1000/1000 [40:34<00:00,  2.43s/it]


#### 3.2 SAVE Dataset

In [ ]:
def save_en_quc_dataset_to_files(dataset: DatasetDict, output_directory: str):
    """
    Saves the English-K'iche' dataset splits (train, test) into separate .en and .quc files.

    Args:
        dataset (DatasetDict): The dataset object containing 'train' and 'test' splits,
                               where each example has 'input' (English) and 'target' (K'iche').
        output_directory (str): The path to the directory where the files will be saved.
                                This directory will be created if it doesn't exist.
    """
    if not isinstance(dataset, DatasetDict):
        print(f"❌ Error: Expected 'dataset' to be a DatasetDict, but got {type(dataset)}")
        return

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    print(f"Ensuring output directory exists: {output_directory}")

    try:
        # Save each split (train, test) into .en (English) and .quc (K'iche') files
        for split in ["train", "test"]:
            if split not in dataset:
                print(f"⚠️ Warning: Split '{split}' not found in the dataset. Skipping.")
                continue

            # Check if 'input' and 'target' keys exist in the first example of the split
            # This is a basic check to ensure the expected structure
            if dataset[split] and ("input" not in dataset[split][0] or "target" not in dataset[split][0]):
                print(f"❌ Error: Expected 'input' and 'target' keys in dataset[{split}] examples. Skipping.")
                continue

            file_path_en = os.path.join(output_directory, f"{split}.en")
            file_path_quc = os.path.join(output_directory, f"{split}.quc")

            with open(file_path_en, "w", encoding="utf-8") as f_en, \
                 open(file_path_quc, "w", encoding="utf-8") as f_quc:

                for example in dataset[split]:
                    f_en.write(example["input"].strip() + "\n")
                    f_quc.write(example["target"].strip() + "\n")

            print(f"✅ Files '{split}.en' and '{split}.quc' saved to {output_directory}")

        print(f"✅ All .en and .quc files processed and saved to {output_directory}")

    except Exception as e:
        print(f"❌ An error occurred while saving the files: {e}")



In [ ]:
save_en_quc_dataset_to_files(en_quc_dataset, EN_QUC_DATASET_FILES_PATH)

Ensuring output directory exists: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files
✅ Files 'train.en' and 'train.quc' saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files
✅ Files 'test.en' and 'test.quc' saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files
✅ All .en and .quc files processed and saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files


In [ ]:
# prompt: save en_quc_dataset to disk (Dataset)

en_quc_dataset.save_to_disk(f"{DRIVE_PATH}/en_quc_dataset")
print(f"✅ Dataset 'en_quc_dataset' saved to disk at: {DRIVE_PATH}/en_quc_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/624 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Dataset 'en_quc_dataset' saved to disk at: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset


#### 3.3 Preparación y entrenamiento

##### Verificamos el dataset

In [70]:
en_quc_dataset["train"][1:10]

{'input': ['The branch is cut',
  '# Quc # In coastal I keep the mazorca',
  'The napkin was pierced by the stove',
  'We must take care of our nervous system',
  '# What # You brought my order',
  "#quc# Today it's expensive to pick up the ground",
  '# Quc # Bread went up in price',
  'Don Juan is blind',
  "# Quc # After tomorrow I'll go with you"],
 'target': ['Ketom ri uqʼabʼ ri cheʼ',
  'Pa taq ri bʼrin kinkʼol wi ri jal',
  'Ri suʼt teqʼtobʼinaq rumal ri rachaq qʼaqʼ',
  'Chqachijij bʼa ri qibʼochil',
  'Xakʼam loq ri wuqxaʼn',
  'Paqal rajil ri joxkʼim che ri ulew',
  'Ri kaxlaʼn wa xpaqiʼ rajil',
  'Ri tat Xwan potzʼ',
  'Kinbʼe awukʼ kabʼij']}

##### Preparacion

In [71]:
model_name = "facebook/nllb-200-distilled-600M"
en_quc_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

base_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

en_quc_model = get_peft_model(base_model, lora_config)

In [72]:
def preprocess_function_en_quc(examples):
    max_length = 128
    inputs = en_quc_tokenizer(examples["input"], max_length=max_length, truncation=True, padding="max_length")
    targets = en_quc_tokenizer(examples["target"], max_length=max_length, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_en_quc_train = en_quc_dataset["train"].map(preprocess_function_en_quc, batched=True)
tokenized_en_quc_test = en_quc_dataset["test"].map(preprocess_function_en_quc, batched=True)

Map:   0%|          | 0/624 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [73]:
training_args_en_quc = TrainingArguments(
    output_dir=f"{DRIVE_PATH}/finetuned_en_quc_nllb",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    logging_dir="./logs_en_quc",
    logging_steps=100,
    report_to="none" # Disable reporting to wandb
)

"""
training_args = TrainingArguments(
    output_dir="./qlora_finetuned_qum_nllb",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs_qlora",
    save_total_limit=2,
    bf16=False,
    fp16=True,
    report_to="none"
)
"""

'\ntraining_args = TrainingArguments(\n    output_dir="./qlora_finetuned_qum_nllb",\n    per_device_train_batch_size=4,\n    per_device_eval_batch_size=4,\n    num_train_epochs=3,\n    learning_rate=2e-4,\n    eval_strategy="epoch",\n    save_strategy="epoch",\n    logging_dir="./logs_qlora",\n    save_total_limit=2,\n    bf16=False,\n    fp16=True,\n    report_to="none"\n)\n'

In [74]:
"""


trainer = Trainer(
    model=finetune_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=finetune_tokenizer,
)
"""


trainer_en_quc = Trainer(
    model=en_quc_model,
    args=training_args_en_quc,
    train_dataset=tokenized_en_quc_train,
    eval_dataset=tokenized_en_quc_test,
    tokenizer=en_quc_tokenizer,
)



trainer_en_quc.train()

# Define the save path for the EN-QUC model
save_path_en_quc = f"{DRIVE_PATH}/finetuned_en_quc_nllb"

# Save the model and tokenizer to the specified path
en_quc_model.save_pretrained(save_path_en_quc)
en_quc_tokenizer.save_pretrained(save_path_en_quc)

print(f"✅ EN-QUC model and tokenizer saved to {save_path_en_quc}")

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,12.843750
2,12.959100,12.585938
3,12.550700,12.343750


✅ EN-QUC model and tokenizer saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/finetuned_en_quc_nllb


## Load from Google Drive

#### Load English Kiche Zero-Shot Model

In [81]:
model_name = "facebook/nllb-200-distilled-600M"
zeroshot_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
zeroshot_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

zeroshot_tokenizer.src_lang = "eng_Latn"
forced_bos_id = zeroshot_tokenizer.convert_tokens_to_ids("quc_Latn")

#### Load Spanish Kiche Model

In [ ]:
def load_models_and_tokenizers_espanol_kiche(drive_path: str, base_model_id: str):
    """
    Loads the fine-tuned Español-K'iche' QLoRA model and tokenizer.

    Args:
        drive_path (str): Base path to your Google Drive.
        base_model_id (str): Identifier for the base model (e.g., "facebook/nllb-200-distilled-600M").

    Returns:
        tuple: A tuple containing reloaded_finetune_model and reloaded_finetune_tokenizer.
               Returns (None, None) if loading fails.
    """
    reloaded_finetune_model = None
    reloaded_finetune_tokenizer = None
    save_path_es_quc = f"{drive_path}/qlora_finetuned_es_quc_nllb"

    if os.path.exists(save_path_es_quc):
        print(f"✅ Found fine-tuned model directory at {save_path_es_quc}")
        try:
            # Load the base model first (needed for QLoRA)
            reloaded_base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_id)

            # Load the QLoRA adapter weights on top of the base model
            reloaded_finetune_model = PeftModel.from_pretrained(reloaded_base_model, save_path_es_quc)

            # Load the tokenizer
            reloaded_finetune_tokenizer = AutoTokenizer.from_pretrained(save_path_es_quc)

            print(f"✅ Model and tokenizer reloaded successfully from {save_path_es_quc}")
        except Exception as e:
            print(f"❌ Error reloading fine-tuned model: {e}")
    else:
        print(f"❌ Fine-tuned model directory not found at {save_path_es_quc}")
        print("Please ensure the model was saved correctly.")

    return reloaded_finetune_model, reloaded_finetune_tokenizer

In [ ]:
# Load models and tokenizers
espanol_kiche_finetune_model, espanol_kiche_finetune_tokenizer = load_models_and_tokenizers_espanol_kiche(DRIVE_PATH, BASE_MODEL_ID)
es_quc_model = espanol_kiche_finetune_model
es_quc_tokenizer = espanol_kiche_finetune_tokenizer

✅ Found fine-tuned model directory at /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/qlora_finetuned_es_quc_nllb
✅ Model and tokenizer reloaded successfully from /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/qlora_finetuned_es_quc_nllb


#### Load English Kiche Model

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # Import AutoModelForSeq2SeqLM

def load_finetuned_english_kiche_model(drive_path: str, base_model_id: str):
    """
    Loads the fine-tuned English-K'iche' model and tokenizer from Google Drive.

    Args:
        drive_path (str): Base path to your Google Drive where the model is saved.
        base_model_id (str): Identifier for the base model (e.g., "facebook/nllb-200-distilled-600M").

    Returns:
        tuple: A tuple containing reloaded_model and reloaded_tokenizer.
               Returns (None, None) if loading fails.
    """
    reloaded_model = None
    reloaded_tokenizer = None
    save_path = f"{drive_path}/finetuned_en_quc_nllb"

    if os.path.exists(save_path):
        print(f"✅ Found fine-tuned model directory at {save_path}")
        try:
            # Load the fine-tuned model directly
            reloaded_model = AutoModelForSeq2SeqLM.from_pretrained(save_path)

            # Load the tokenizer
            reloaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

            print(f"✅ Model and tokenizer reloaded successfully from {save_path}")
        except Exception as e:
            print(f"❌ Error reloading fine-tuned model: {e}")
            reloaded_model = None
            reloaded_tokenizer = None
    else:
        print(f"❌ Fine-tuned model directory not found at {save_path}")
        print("Please ensure the model was saved correctly.")

    return reloaded_model, reloaded_tokenizer

# Example usage (you can run this cell to test loading)
# en_quc_finetuned_model, en_quc_finetuned_tokenizer = load_finetuned_english_kiche_model(DRIVE_PATH, BASE_MODEL_ID)

In [ ]:
english_kiche_finetuned_model, english_kiche_finetuned_tokenizer = load_finetuned_english_kiche_model(DRIVE_PATH, BASE_MODEL_ID)
en_quc_model = english_kiche_finetuned_model
en_quc_tokenizer = english_kiche_finetuned_tokenizer

✅ Found fine-tuned model directory at /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/finetuned_en_quc_nllb
✅ Model and tokenizer reloaded successfully from /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/finetuned_en_quc_nllb


#### Load Spanish-Kiche and English-Kiche datasets

In [63]:
def load_datasets(es_quc_dataset_path: str, en_quc_dataset_files_path: str, mayanv_repo_path: str):
    """
    Carga ambos datasets desde sus respectivas carpetas guardadas en Google Drive.
    """
    loaded_es_quc_dataset = None
    en_quc_dataset = None

    # --- 1. Cargar el dataset Español-K'iche' desde su carpeta guardada ---
    # Se mantiene tu lógica original de cargar desde el disco.
    print(f"\n--- Intentando cargar es_quc_dataset desde: {es_quc_dataset_path} ---")
    try:
        loaded_es_quc_dataset = DatasetDict.load_from_disk(es_quc_dataset_path)
        print("✅ Dataset Español-K'iche' cargado exitosamente desde el disco.")
        print(loaded_es_quc_dataset)
    except Exception as e:
        print(f"❌ Error al cargar es_quc_dataset desde el disco: {e}")
        print("Asegúrate de haber ejecutado la celda que lo guarda con 'save_to_disk' primero.")

    # --- 2. Cargar el dataset sintético Inglés-K'iche' desde archivos de texto ---
    # Aquí aplicamos la corrección para que se cargue como un dataset paralelo.
    print(f"\n--- Cargando dataset sintético desde: {en_quc_dataset_files_path} ---")

    def load_parallel_corpus_from_files(split_name):
        """Función interna para leer los archivos .en y .quc y unirlos."""
        source_path = f"{en_quc_dataset_files_path}/{split_name}.en"
        target_path = f"{en_quc_dataset_files_path}/{split_name}.quc"

        if not os.path.exists(source_path) or not os.path.exists(target_path):
            print(f"ADVERTENCIA: Archivos para el split '{split_name}' no encontrados en la ruta.")
            return None

        with open(source_path, "r", encoding="utf-8") as f:
            source_lines = [line.strip() for line in f.readlines()]
        with open(target_path, "r", encoding="utf-8") as f:
            target_lines = [line.strip() for line in f.readlines()]

        if len(source_lines) != len(target_lines):
            raise ValueError(f"Archivos del split '{split_name}' no tienen el mismo número de líneas.")

        # Devolvemos un Dataset con las columnas "input" y "target"
        return Dataset.from_dict({"input": source_lines, "target": target_lines})

    train_split = load_parallel_corpus_from_files("train")
    test_split = load_parallel_corpus_from_files("test")

    if train_split and test_split:
        en_quc_dataset = DatasetDict({"train": train_split, "test": test_split})
        print("✅ Dataset sintético Inglés-K'iche' cargado correctamente con 2 columnas.")
    else:
        print("❌ Fallo al cargar el dataset sintético.")
        en_quc_dataset = None

    return loaded_es_quc_dataset, en_quc_dataset

In [64]:
# Load datasets and verify repository
espanol_kiche_dataset, english_kiche_dataset = load_datasets(ES_QUC_DATASET_PATH, EN_QUC_DATASET_FILES_PATH, MAYANV_REPO_PATH)
es_quc_dataset = espanol_kiche_dataset
en_quc_dataset = english_kiche_dataset


--- Intentando cargar es_quc_dataset desde: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/es_quc_dataset ---
✅ Dataset Español-K'iche' cargado exitosamente desde el disco.
DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 624
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 1000
    })
})

--- Cargando dataset sintético desde: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files ---
✅ Dataset sintético Inglés-K'iche' cargado correctamente con 2 columnas.


In [65]:
for i in range(0, 20):
    print(f"> Input:     {es_quc_dataset['test'][i]['input']}")
    print(f"> Target:  {es_quc_dataset['test'][i]['target']}")
    print("-")

> Input:     Kqʼulqʼut ri apam
> Target:  #quc# Tienes indigestión
-
> Input:     Xanimaj bʼik ri kuk
> Target:  #quc# La ardilla se escapó
-
> Input:     Tajin kamuxan ri awecham
> Target:  #quc# Tu cuñado está nadando
-
> Input:     Xinriq raxtew pa taqʼaj
> Target:  #quc# En la costa me contagié de paludismo
-
> Input:     Xinpe rukʼ chech ubʼanik siʼ, pa ri juyubʼ
> Target:  #quc# Vengo con él para hacer leña en la montaña
-
> Input:     Kʼo kʼaqat chi raqan ri tataʼ
> Target:  #quc# El señor tiene picazón en los pies
-
> Input:     Rajwaxik kawetaʼmaj pa uwiʼ ri chʼajchʼojisanem
> Target:  #quc# Necesitas aprender acerca el aseo
-
> Input:     Nim ri qʼoqʼ
> Target:  #quc# El chilacayote es grande
-
> Input:     Chqʼalajisaj awibʼ chi kiwach ri tinamit
> Target:  #quc# Aclárate ante el pueblo
-
> Input:     Kakowij che ri apatan
> Target:  #quc# Te apuras a tus tareas
-
> Input:     Ri abʼaj yatzalik
> Target:  #quc# La piedra es liviana
-
> Input:     Pa juyubʼ Sibʼalaj raqan neʼ

In [66]:
for i in range(0, 20):
    print(f"> Input:     {en_quc_dataset['test'][i]['input']}")
    print(f"> Target:  {en_quc_dataset['test'][i]['target']}")
    print("-")

> Input:     You have indigestion
> Target:  Kqʼulqʼut ri apam
-
> Input:     The squirrel escaped
> Target:  Xanimaj bʼik ri kuk
-
> Input:     # What's up? # Your brother-in-law is swimming
> Target:  Tajin kamuxan ri awecham
-
> Input:     #quc# On the coast I got malaria
> Target:  Xinriq raxtew pa taqʼaj
-
> Input:     #quc# I'm coming with him to make wood in the mountain
> Target:  Xinpe rukʼ chech ubʼanik siʼ, pa ri juyubʼ
-
> Input:     The gentleman has itching feet
> Target:  Kʼo kʼaqat chi raqan ri tataʼ
-
> Input:     #quc# You need to learn about the toilet
> Target:  Rajwaxik kawetaʼmaj pa uwiʼ ri chʼajchʼojisanem
-
> Input:     The chilacayote is big
> Target:  Nim ri qʼoqʼ
-
> Input:     # Quc # Clear yourself up to the people
> Target:  Chqʼalajisaj awibʼ chi kiwach ri tinamit
-
> Input:     # Quc# You rush to your homework
> Target:  Kakowij che ri apatan
-
> Input:     ♪ The stone is light ♪
> Target:  Ri abʼaj yatzalik
-
> Input:     # Quc # In the mountains there 

In [68]:
en_quc_dataset["test"][1]

{'input': 'The squirrel escaped', 'target': 'Xanimaj bʼik ri kuk'}

## Evaluación Comparativa de Experimentos

### Métricas

In [69]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


### Experimentos

#### 1. Zero-shot

Como NLLB tiene el idioma K’iche’, se coloca este como idioma objetivo para las traducciones directas.

In [113]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# --- PARTE CLAVE: Define los códigos de idioma correctos ---
# Estos son los códigos que NLLB-200 entiende.
SOURCE_LANG_CODE = "eng_Latn"
TARGET_LANG_CODE = "quc_Latn"

# Tu código para cargar el modelo y el tokenizador (está correcto)
model_name = "facebook/nllb-200-distilled-600M"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Cargando modelo y tokenizador en el dispositivo: {device}")
zeroshot_tokenizer = AutoTokenizer.from_pretrained(model_name)
zeroshot_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
zeroshot_model.to(device)

print("✅ Modelo Zero-Shot listo y en el dispositivo correcto.")

def generate_predictions(model_to_test, tokenizer_to_use, dataset_to_test, n_examples=1000):
    predictions = []
    references = []

    model_to_test.eval()
    device = model_to_test.device

    # --- CORRECCIÓN ---
    # 1. Configurar los idiomas en el tokenizador ANTES del bucle.
    # 2. Usar los códigos correctos "eng_Latn" y "quc_Latn".
    tokenizer_to_use.src_lang = SOURCE_LANG_CODE
    forced_bos_id = tokenizer_to_use.convert_tokens_to_ids(TARGET_LANG_CODE)

    # Verifica que el forced_bos_id sea válido
    if forced_bos_id is None:
        raise ValueError(f"El código de idioma de destino '{TARGET_LANG_CODE}' no es válido para este tokenizador.")

    print(f"Iniciando predicciones de '{SOURCE_LANG_CODE}' a '{TARGET_LANG_CODE}' para {n_examples} ejemplos...")
    for item in tqdm(dataset_to_test.select(range(n_examples))):
        input_text = item["input"]
        reference = item["target"]

        inputs = tokenizer_to_use(input_text, return_tensors="pt", truncation=True).to(device)

        with torch.no_grad():
            outputs = model_to_test.generate(
                **inputs,
                forced_bos_token_id=forced_bos_id, # Esta es la instrucción clave para el idioma de salida
                max_length=128
            )

        pred = tokenizer_to_use.batch_decode(outputs, skip_special_tokens=True)[0]

        predictions.append(pred)
        references.append(reference)

    return predictions, references

# --- Llamada a la función (no necesita cambios) ---
# Asegúrate de que 'cleaned_en_quc_dataset' esté disponible
test_dataset_final = cleaned_en_quc_dataset["test"]

predictions, references = generate_predictions(
    model_to_test=zeroshot_model,
    tokenizer_to_use=zeroshot_tokenizer,
    dataset_to_test=test_dataset_final,
    n_examples=10
)

# --- Impresión de resultados (no necesita cambios) ---
print("\n=== Ejemplos ===\n")
for i in range(10):
    print(f"> Entrada:    {cleaned_en_quc_dataset['test'][i]['input']}")
    print(f"> Referencia: {references[i]}")
    print(f"> Predicción: {predictions[i]}\n")

Cargando modelo y tokenizador en el dispositivo: cuda
✅ Modelo Zero-Shot listo y en el dispositivo correcto.
Iniciando predicciones de 'eng_Latn' a 'quc_Latn' para 10 ejemplos...


100%|██████████| 10/10 [00:06<00:00,  1.64it/s]


=== Ejemplos ===

> Entrada:    You have indigestion
> Referencia: Kqʼulqʼut ri apam
> Predicción: You have indigestion

> Entrada:    The squirrel escaped
> Referencia: Xanimaj bʼik ri kuk
> Predicción: The squirrel escaped

> Entrada:    What's up?  Your brother-in-law is swimming
> Referencia: Tajin kamuxan ri awecham
> Predicción: What's up?Your brother-in-law is swimming

> Entrada:    On the coast I got malaria
> Referencia: Xinriq raxtew pa taqʼaj
> Predicción: ️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️

> Entrada:    I'm coming with him to make wood in the mountain
> Referencia: Xinpe rukʼ chech ubʼanik siʼ, pa ri juyubʼ
> Predicción: I'm coming with him to make wood in the mountain

> Entrada:    The gentleman has itching feet
> Referencia: Kʼo kʼaqat chi raqan ri tataʼ
> Predicción: The gentleman has itching feet

> Entrada:    You need to learn about the toilet
> Referencia: Rajwaxik kawetaʼ

In [123]:
# --- PRUEBA DE SANIDAD ---
# Cambiamos temporalmente el idioma de destino a Italiano
TARGET_LANG_CODE_TEST = "ita_Latn"
SOURCE_LANG_CODE_TEST = "eng_Latn"

# Usamos tu mismo tokenizador y modelo
zeroshot_tokenizer.src_lang = SOURCE_LANG_CODE_TEST
forced_bos_id_test = zeroshot_tokenizer.convert_tokens_to_ids(TARGET_LANG_CODE_TEST)

print(f"ID del token de destino (Italiano): {forced_bos_id_test}") # Debería imprimir un número ej: 256116

# Tomemos una sola frase para la prueba
input_text_test = "The squirrel escaped"
inputs_test = zeroshot_tokenizer(input_text_test, return_tensors="pt").to(device)

print("\nRealizando prueba de sanidad (Inglés -> Italiano)...")
with torch.no_grad():
    outputs_test = zeroshot_model.generate(
        **inputs_test,
        forced_bos_token_id=forced_bos_id_test,
        max_length=128
    )

prediction_test = zeroshot_tokenizer.batch_decode(outputs_test, skip_special_tokens=True)[0]

print(f"\nEntrada: '{input_text_test}'")
print(f"Predicción en Italiano: '{prediction_test}'") # <-- ¿Qué imprime aquí?


from transformers import AutoTokenizer

# Cargamos el tokenizador oficial de NLLB
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# --- Verifiquemos los idiomas ---

kiche_code = "quc_Latn"
sipakapense_code = "qum_Latn"

print(f"Buscando el ID para K'iche' ({kiche_code})...")
try:
    kiche_id = tokenizer.convert_tokens_to_ids(kiche_code)
    print(f"✅ ÉXITO: El ID para '{kiche_code}' es: {kiche_id}. ¡SÍ EXISTE!")
except Exception as e:
    print(f"❌ FALLO: No se pudo encontrar un ID para '{kiche_code}'. ¡NO EXISTE!")

print("-" * 20)

print(f"Buscando el ID para Sipakapense ({sipakapense_code})...")
try:
    # El tokenizador arroja un error si no encuentra el token.
    # Usamos .get para manejarlo sin que el programa se caiga.
    sipakapense_id = tokenizer.vocab.get(sipakapense_code)
    if sipakapense_id is not None:
         print(f"✅ ÉXITO: El ID para '{sipakapense_code}' es: {sipakapense_id}. ¡SÍ EXISTE!")
    else:
         print(f"❌ FALLO: No se encontró un ID para '{sipakapense_code}'. ¡NO EXISTE!")
except Exception as e:
    print(f"❌ FALLO: Ocurrió un error buscando '{sipakapense_code}'.")


# --- PRUEBA DE SANIDAD ---
# Cambiamos temporalmente el idioma de destino a Kiche
TARGET_LANG_CODE_TEST = "quc_Latn"
SOURCE_LANG_CODE_TEST = "spa_Latn"

# Usamos tu mismo tokenizador y modelo
zeroshot_tokenizer.src_lang = SOURCE_LANG_CODE_TEST
forced_bos_id_test = zeroshot_tokenizer.convert_tokens_to_ids(TARGET_LANG_CODE_TEST)

print(f"ID del token de destino (Kiche): {forced_bos_id_test}") # Debería imprimir un número ej: 256116

# Tomemos una sola frase para la prueba
input_text_test = "La ardilla escapó el otro dia"
inputs_test = zeroshot_tokenizer(input_text_test, return_tensors="pt").to(device)

print("\nRealizando prueba de sanidad (Español -> Kiche)...")
with torch.no_grad():
    outputs_test = zeroshot_model.generate(
        **inputs_test,
        forced_bos_token_id=forced_bos_id_test,
        max_length=128
    )

prediction_test = zeroshot_tokenizer.batch_decode(outputs_test, skip_special_tokens=True)[0]

print(f"\nEntrada: '{input_text_test}'")
print(f"Predicción en Kiche: '{prediction_test}'") # <-- ¿Qué imprime aquí?


ID del token de destino (Italiano): 256077

Realizando prueba de sanidad (Inglés -> Italiano)...

Entrada: 'The squirrel escaped'
Predicción en Italiano: 'L' scoiattolo è scappato .'
Buscando el ID para K'iche' (quc_Latn)...
✅ ÉXITO: El ID para 'quc_Latn' es: 3. ¡SÍ EXISTE!
--------------------
Buscando el ID para Sipakapense (qum_Latn)...
❌ FALLO: No se encontró un ID para 'qum_Latn'. ¡NO EXISTE!
ID del token de destino (Kiche): 3

Realizando prueba de sanidad (Español -> Kiche)...

Entrada: 'La ardilla escapó el otro dia'
Predicción en Kiche: 'The squirrel escaped the other day'


In [100]:
import re

def clean_english_input(example):
    """
    Toma un ejemplo del dataset y limpia únicamente la columna 'input' (inglés),
    eliminando los artefactos y etiquetas no deseadas.
    """
    # Patrones a eliminar: #quc# (y variantes), #, ♪
    dirty_text = example["input"]

    # Usamos expresiones regulares para una limpieza robusta
    # Elimina #quc#, # quc #, # Quc #, etc. ignorando mayúsculas/minúsculas y espacios
    clean_text = re.sub(r'#\s*quc\s*#', '', dirty_text, flags=re.IGNORECASE)
    # Elimina el símbolo ♪
    clean_text = re.sub(r'♪', '', clean_text)
    # Elimina cualquier caracter # que haya quedado suelto
    clean_text = clean_text.replace('#', '')

    # Elimina espacios extra al principio o al final
    example["input"] = clean_text.strip()

    return example

# --- Aplicar la función de limpieza a todo el dataset ---
# .map() es una forma muy eficiente de aplicar una función a cada fila
print("🧼 Limpiando el dataset sintético 'en_quc_dataset'...")

cleaned_en_quc_dataset = en_quc_dataset.map(
    clean_english_input,
    desc="Limpiando columna 'input'"
)

print("\n✅ Limpieza completada.")

# --- Verificación ---
# Imprimamos un par de ejemplos para ver el ANTES y el DESPUÉS.
print("\n--- Verificando la limpieza ---")

# Ejemplo problemático ANTES de la limpieza
original_example = en_quc_dataset["test"][2]
print(f"Original \t> Entrada: {original_example['input']}")
print(f"\t\t> Referencia: {original_example['target']}")

# Mismo ejemplo DESPUÉS de la limpieza
cleaned_example = cleaned_en_quc_dataset["test"][2]
print(f"Limpiado \t> Entrada: {cleaned_example['input']}")
print(f"\t\t> Referencia: {cleaned_example['target']}")

# Otro ejemplo
original_example_2 = en_quc_dataset["test"][3]
print(f"\nOriginal \t> Entrada: {original_example_2['input']}")
print(f"\t\t> Referencia: {original_example_2['target']}")

cleaned_example_2 = cleaned_en_quc_dataset["test"][3]
print(f"Limpiado \t> Entrada: {cleaned_example_2['input']}")
print(f"\t\t> Referencia: {cleaned_example_2['target']}")

🧼 Limpiando el dataset sintético 'en_quc_dataset'...


Limpiando columna 'input':   0%|          | 0/624 [00:00<?, ? examples/s]

Limpiando columna 'input':   0%|          | 0/1000 [00:00<?, ? examples/s]


✅ Limpieza completada.

--- Verificando la limpieza ---
Original 	> Entrada: # What's up? # Your brother-in-law is swimming
		> Referencia: Tajin kamuxan ri awecham
Limpiado 	> Entrada: What's up?  Your brother-in-law is swimming
		> Referencia: Tajin kamuxan ri awecham

Original 	> Entrada: #quc# On the coast I got malaria
		> Referencia: Xinriq raxtew pa taqʼaj
Limpiado 	> Entrada: On the coast I got malaria
		> Referencia: Xinriq raxtew pa taqʼaj


In [104]:
for i in range(1000):
    print("//")
    print(f"> Entrada:     {cleaned_en_quc_dataset['test'][i]['input']}")
    print(f"> Referencia:  {cleaned_en_quc_dataset['test'][i]['target']}")
    print("--")

//
> Entrada:     You have indigestion
> Referencia:  Kqʼulqʼut ri apam
--
//
> Entrada:     The squirrel escaped
> Referencia:  Xanimaj bʼik ri kuk
--
//
> Entrada:     What's up?  Your brother-in-law is swimming
> Referencia:  Tajin kamuxan ri awecham
--
//
> Entrada:     On the coast I got malaria
> Referencia:  Xinriq raxtew pa taqʼaj
--
//
> Entrada:     I'm coming with him to make wood in the mountain
> Referencia:  Xinpe rukʼ chech ubʼanik siʼ, pa ri juyubʼ
--
//
> Entrada:     The gentleman has itching feet
> Referencia:  Kʼo kʼaqat chi raqan ri tataʼ
--
//
> Entrada:     You need to learn about the toilet
> Referencia:  Rajwaxik kawetaʼmaj pa uwiʼ ri chʼajchʼojisanem
--
//
> Entrada:     The chilacayote is big
> Referencia:  Nim ri qʼoqʼ
--
//
> Entrada:     Clear yourself up to the people
> Referencia:  Chqʼalajisaj awibʼ chi kiwach ri tinamit
--
//
> Entrada:     You rush to your homework
> Referencia:  Kakowij che ri apatan
--
//
> Entrada:     The stone is light
> Referenc

In [94]:
n_examples = 10

In [112]:
# Mostrar algunas predicciones
print("\n=== Ejemplos ===\n")
for i in range(n_examples):
    print(f"> Entrada:     {cleaned_en_quc_dataset['test'][i]['input']}")
    print(f"> Referencia:  {cleaned_en_quc_dataset['test'][i]['target']}")
    print(f"> Predicción:  {predictions[i]}\n")


=== Ejemplos ===

> Entrada:     You have indigestion
> Referencia:  Kqʼulqʼut ri apam
> Predicción:  You have indigestion

> Entrada:     The squirrel escaped
> Referencia:  Xanimaj bʼik ri kuk
> Predicción:  The squirrel escaped

> Entrada:     What's up?  Your brother-in-law is swimming
> Referencia:  Tajin kamuxan ri awecham
> Predicción:  What's up?Your brother-in-law is swimming

> Entrada:     On the coast I got malaria
> Referencia:  Xinriq raxtew pa taqʼaj
> Predicción:  ️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️

> Entrada:     I'm coming with him to make wood in the mountain
> Referencia:  Xinpe rukʼ chech ubʼanik siʼ, pa ri juyubʼ
> Predicción:  I'm coming with him to make wood in the mountain

> Entrada:     The gentleman has itching feet
> Referencia:  Kʼo kʼaqat chi raqan ri tataʼ
> Predicción:  The gentleman has itching feet

> Entrada:     You need to learn about the toilet
> Referenci

In [ ]:
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

print("\n=== Métricas BLEU ===")
print(f"BLEU-1: {bleu_score['precisions'][0]*100:.2f}")
print(f"BLEU-2: {bleu_score['precisions'][1]*100:.2f}")
print(f"BLEU-3: {bleu_score['precisions'][2]*100:.2f}")
print(f"BLEU-4: {bleu_score['precisions'][3]*100:.2f}")
print(f"BLEU total: {bleu_score['bleu']*100:.2f}")

rouge_score = rouge.compute(predictions=predictions, references=references)

print("\n=== Métricas ROUGE ===")
print(f"ROUGE-1:   {rouge_score['rouge1']*100:.2f}")
print(f"ROUGE-2:   {rouge_score['rouge2']*100:.2f}")
print(f"ROUGE-L:   {rouge_score['rougeL']*100:.2f}")
print(f"ROUGE-Lsum:{rouge_score['rougeLsum']*100:.2f}")


=== Métricas BLEU ===
BLEU-1: 1.03
BLEU-2: 0.07
BLEU-3: 0.00
BLEU-4: 0.00
BLEU total: 0.00

=== Métricas ROUGE ===
ROUGE-1:   0.74
ROUGE-2:   0.08
ROUGE-L:   0.72
ROUGE-Lsum:0.73


In [ ]:
with open(f"{DRIVE_PATH}/results_es_quc.tsv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["input", "reference", "prediction"])

    for example, pred, ref in zip(es_quc_dataset["test"], predictions, references):
        writer.writerow([example["input"], ref, pred])

print("✅ Resultados guardados en 'results_es_quc.tsv'")

✅ Resultados guardados en 'results_es_quc.tsv'


#### 2. Finetuning ES-QUC y Pipeline

In [ ]:
predictions_intermediate = []
references_intermediate = []

print("\n🔹 Traduciendo: inglés → español → K’iche’ (vía intermediaria)\n")

for item in tqdm(es_quc_dataset["test"]):
    input_text = item["input"]
    reference = item["target"]

    # Paso 1: inglés → español (modelo base)
    inter = translate_text(zeroshot_model, zeroshot_tokenizer, input_text, "eng_Latn", "spa_Latn")

    # Paso 2: español → K’iche’ (modelo fine-tuneado)
    pred = translate_text(finetune_model, finetune_tokenizer, inter, "spa_Latn", "quc_Latn")

    predictions_intermediate.append(pred)
    references_intermediate.append(reference)


🔹 Traduciendo: inglés → español → K’iche’ (vía intermediaria)



  0%|          | 0/1000 [00:02<?, ?it/s]


NameError: name 'finetune_model' is not defined

In [ ]:
print("\n=== Ejemplos Fine-Tuned ===\n")
for i in range(5):
    print(f"> Entrada:     {es_quc_dataset['test'][i]['input']}")
    print(f"> Referencia:  {es_quc_dataset['test'][i]['target']}")
    print(f"> Predicción:  {predictions_intermediate[i]}\n")

In [ ]:
bleu_score_ft = bleu.compute(predictions=predictions_intermediate, references=[[r] for r in references_intermediate])
rouge_score_ft = rouge.compute(predictions=predictions_intermediate, references=references_intermediate)

print("\n=== Métricas Fine-Tuned ===")
print(f"BLEU: {bleu_score_ft['bleu']:.2f}")
print(f"ROUGE-L: {rouge_score_ft['rougeL']:.2f}")

In [ ]:
# prompt: Save the relevant results, predictions, etc

# Assuming predictions_intermediate and references_intermediate are already computed from the previous cell

output_filename_ft = f"{DRIVE_PATH}/results_finetuned_es_quc_pipeline.tsv"

with open(output_filename_ft, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    # Write header
    writer.writerow(["input_english", "intermediate_spanish", "reference_kiche", "prediction_kiche"])

    # Write data
    # Need to regenerate the intermediate Spanish translations to save them
    # or store them during the generation process
    intermediate_translations = []
    print("\n🔄 Regenerating intermediate Spanish translations for saving...")
    for item in tqdm(es_quc_dataset["test"]):
         input_text = item["input"]
         inter = translate_text(zeroshot_model, zeroshot_tokenizer, input_text, "eng_Latn", "spa_Latn")
         intermediate_translations.append(inter)


    for i in range(len(es_quc_dataset["test"])):
         writer.writerow([
              es_quc_dataset["test"][i]["input"], # Original English input from dataset
              intermediate_translations[i], # Intermediate Spanish translation
              references_intermediate[i], # Original K'iche' reference from dataset
              predictions_intermediate[i] # Final K'iche' prediction from the fine-tuned model
         ])

print(f"✅ Resultados del pipeline (inglés -> español -> K'iche') guardados en '{output_filename_ft}'")


#### 3. Finetuning ES-QUC y Traducción Directa

In [ ]:
predictions_direct = []
references_direct = []

print("\n🔹 Traduciendo: inglés → K’iche’ (directo)\n")

for item in tqdm(es_quc_dataset["test"]):
    input_text = item["input"]
    reference = item["target"]
    pred = translate_text(finetune_model, finetune_tokenizer, input_text, "eng_Latn", "quc_Latn")

    predictions_direct.append(pred)
    references_direct.append(reference)

In [ ]:
print("\n=== Ejemplos Fine-Tuned ===\n")
for i in range(5):
    print(f"> Entrada:     {es_quc_dataset['test'][i]['input']}")
    print(f"> Referencia:  {es_quc_dataset['test'][i]['target']}")
    print(f"> Predicción:  {predictions_direct[i]}\n")

In [ ]:
bleu_score_ft = bleu.compute(predictions=predictions_direct, references=[[r] for r in references_direct])
rouge_score_ft = rouge.compute(predictions=predictions_direct, references=references_direct)

print("\n=== Métricas Fine-Tuned ===")
print(f"BLEU: {bleu_score_ft['bleu']:.2f}")
print(f"ROUGE-L: {rouge_score_ft['rougeL']:.2f}")

In [ ]:
# prompt: Save the relevant results

# Assuming predictions_direct and references_direct are already computed from the previous cell

output_filename_direct = f"{DRIVE_PATH}/results_finetuned_es_quc_direct.tsv"

with open(output_filename_direct, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    # Write header
    writer.writerow(["input_english", "reference_kiche", "prediction_kiche"])

    # Write data
    for i in range(len(es_quc_dataset["test"])):
         writer.writerow([
              es_quc_dataset["test"][i]["input"], # Original English input from dataset
              references_direct[i], # Original K'iche' reference from dataset
              predictions_direct[i] # Final K'iche' prediction from the fine-tuned model
         ])

print(f"✅ Resultados de traducción directa (inglés -> K'iche') guardados en '{output_filename_direct}'")


#### 4. Finetuning EN-QUC

In [ ]:
predictions_en_qum = []
references_en_qum = []

print("\n🔹 Traduciendo: inglés → K’iche’ (dataset sintético directo)\n")

for item in tqdm(en_qum_dataset["test"]):
    input_text = item["input"]
    reference = item["target"]
    pred = translate_text(en_qum_model, en_qum_tokenizer, input_text, "eng_Latn", "quc_Latn")

    predictions_en_qum.append(pred)
    references_en_qum.append(reference)

In [ ]:


print("\n=== Ejemplos Fine-Tuned ===\n")
for i in range(5):
    print(f"> Entrada:     {en_qum_dataset['test'][i]['input']}")
    print(f"> Referencia:  {en_qum_dataset['test'][i]['target']}")
    print(f"> Predicción:  {predictions_en_qum[i]}\n")



In [ ]:


bleu_score = bleu.compute(predictions=predictions_en_qum, references=[[ref] for ref in references_en_qum])

print("\n=== Métricas BLEU ===")
print(f"BLEU-1: {bleu_score['precisions'][0]*100:.2f}")
print(f"BLEU-2: {bleu_score['precisions'][1]*100:.2f}")
print(f"BLEU-3: {bleu_score['precisions'][2]*100:.2f}")
print(f"BLEU-4: {bleu_score['precisions'][3]*100:.2f}")
print(f"BLEU total: {bleu_score['bleu']*100:.2f}")

rouge_score = rouge.compute(predictions=predictions_en_qum, references=references_en_qum)

print("\n=== Métricas ROUGE ===")
print(f"ROUGE-1:   {rouge_score['rouge1']*100:.2f}")
print(f"ROUGE-2:   {rouge_score['rouge2']*100:.2f}")
print(f"ROUGE-L:   {rouge_score['rougeL']*100:.2f}")
print(f"ROUGE-Lsum:{rouge_score['rougeLsum']*100:.2f}")

# meteor_score = meteor.compute(predictions=predictions_en_qum, references=references_en_qum)

# print("\n=== Otras Métricas ===")
# print(f"METEOR:     {meteor_score['meteor']*100:.2f}")



In [ ]:


filename = "results_en_qum_directo.tsv"
with open(filename, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["input", "reference", "prediction"])

    for example, pred, ref in zip(es_qum_dataset["test"], predictions_en_qum, references_en_qum):
        writer.writerow([example["input"], ref, pred])

print(f"✅ Resultados guardados en '{filename}'")

